In [1]:
!ls -lh data/

total 757M
-rw-r--r-- 1 RENT 197121 379M ׳“׳¦׳� 11 16:36 fixed.csv
-rw-r--r-- 1 RENT 197121 379M ׳“׳¦׳� 11 15:43 oddetect.csv


# Loading The CSV File

In [2]:
%matplotlib inline
import pandas as pd

In [3]:
%time df = pd.read_csv('data/fixed.csv',names=["frame", "x", "y", "obj", "size", "seq", "tbd1", "tbd2", "tbd3", "filename", "start", "path_time", "delta_time", "tbd4"], usecols=["frame", "x", "y", "obj", "size", "seq", "filename", "start", "path_time", "delta_time"])


Wall time: 15.2 s


In [4]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997979 entries, 0 to 2997978
Data columns (total 10 columns):
frame         int64
x             int64
y             int64
obj           int64
size          int64
seq           int64
filename      object
start         object
path_time     object
delta_time    object
dtypes: int64(6), object(4)
memory usage: 671.8 MB


In [5]:
df.describe()


,frame,x,y,obj,size,seq
count,2.997979e+06,2.997979e+06,2.997979e+06,2.997979e+06,2.997979e+06,2.997979e+06
mean,8.289598e+03,2.730162e+02,1.480890e+02,5.053650e+01,4.675812e+07,1.158957e+02
std,5.189150e+03,1.609973e+02,7.650773e+01,8.449936e+01,3.391170e+07,1.502103e+02
min,0.000000e+00,3.000000e+00,3.000000e+00,0.000000e+00,3.620000e+02,0.000000e+00
25%,3.712000e+03,1.440000e+02,9.500000e+01,1.200000e+01,1.419073e+07,1.600000e+01
50%,8.119000e+03,2.690000e+02,1.340000e+02,2.800000e+01,4.510348e+07,5.900000e+01
75%,1.263600e+04,4.020000e+02,2.050000e+02,5.100000e+01,5.399656e+07,1.620000e+02
max,1.847600e+04,6.370000e+02,3.570000e+02,6.430000e+02,1.265195e+08,1.977000e+03


In [6]:
%%time
df_obj = df.select_dtypes(include = ['object']).copy()
converted_obj = pd.DataFrame()

for col in df_obj.columns:
        num_unique_values = len(df_obj[col].unique())
        num_total_values = len(df_obj[col])
        if num_unique_values / num_total_values < 0.5:
            converted_obj.loc[:,col] = df_obj[col].astype('category')
        else:
            converted_obj.loc[:,col] = df_obj[col]


compare_obj = pd.concat([df_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)
df[converted_obj.columns] = converted_obj


Wall time: 4.03 s


In [7]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997979 entries, 0 to 2997978
Data columns (total 10 columns):
frame         int64
x             int64
y             int64
obj           int64
size          int64
seq           int64
filename      category
start         category
path_time     category
delta_time    category
dtypes: category(4), int64(6)
memory usage: 168.4 MB


In [8]:
df_int = df.select_dtypes(include=['int64'])
converted_int = df_int.apply(pd.to_numeric,downcast='unsigned')
df[converted_int.columns] = converted_int

In [9]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997979 entries, 0 to 2997978
Data columns (total 10 columns):
frame         uint16
x             uint16
y             uint16
obj           uint16
size          uint32
seq           uint16
filename      category
start         category
path_time     category
delta_time    category
dtypes: category(4), uint16(5), uint32(1)
memory usage: 71.2 MB


In [10]:
%%time
duplicate_bool = df.duplicated(subset=None, keep='first')
df = df.loc[duplicate_bool == False]
df

Wall time: 3.3 s


In [11]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2997821 entries, 0 to 2997820
Data columns (total 10 columns):
frame         uint16
x             uint16
y             uint16
obj           uint16
size          uint32
seq           uint16
filename      category
start         category
path_time     category
delta_time    category
dtypes: category(4), uint16(5), uint32(1)
memory usage: 94.1 MB


In [12]:
%time df.to_pickle('data/to_pickle.pk1', compression='xz')

Wall time: 51.2 s


In [18]:
%time unpickled_df = pd.read_pickle("/to_pickle.pk1")

KeyError: 253